In [1]:
# from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from time import time
import mxnet as mx
import d2lzh as d2l
import collections
import random
import math

# 10. ⾃然语⾔处理
⾃然语⾔处理关注计算机与⼈类之间的⾃然语⾔交互。在实际中，我们常常使⽤⾃然语⾔处理技术，如“循环神经⽹络”⼀章中介绍的语⾔模型，来处理和分析⼤量的⾃然语⾔数据。本章中，根据输⼊与输出的不同形式，我们按`定⻓到定⻓`、`不定⻓到定⻓`、`不定⻓到不定⻓`的顺序，逐步展⽰在⾃然语⾔处理中如何表征并变换定⻓的词或类别以及不定⻓的句⼦或段落序列：
+ 首先介绍如何⽤向量表⽰词，并在语料库上训练词向量
+ 之后，我们把在更⼤语料库上预训练的词向量应⽤于求近义词和类⽐词，即`定⻓到定⻓`
+ 最后，在⽂本分类这种`不定⻓到定⻓”`的任务中，我们进⼀步应⽤词向量来分析⽂本情感，并分别基于RNN和CNN为表征时序数据提供两种思路

此外，⾃然语⾔处理任务中很多输出是不定⻓的，如任意⻓度的句⼦或段落。我们将描述应对这类问题的编码器—解码器模型、束搜索和注意⼒机制，并动⼿实践`不定⻓到不定⻓`的机器翻译任务。

## 10.1 词嵌入(word2vec)
自然语言是一套用来表达含义的复杂系统。在这套系统中，词是表义的基本单元。顾名思义，词向量是用来表示词的向量，也可被认为是词的特征向量或表征。把词映射为实数域向量的技术也叫`词嵌入`(word embedding)。近年来，词嵌入已逐渐成为自然语言处理的基础知识。


### 10.1.1 为何不采用one-hot向量
我们在`循环神经网络的从零开始实现`一节中使用one-hot向量表示词(字符为词)。假设词典中不同词的数量为$N$，每个词可以和从0到$N-1$的连续整数一一对应。这些与词对应的整数叫作词的索引。

假设一个词的索引为$i$，为了得到该词的one-hot向量表示，我们创建一个全0的长为$N$的向量，并将其第$i$位设成1。这样一来，每个词就表示成了一个长度为$N$的向量，可以直接被神经网络使用。

虽然one-hot词向量构造起来很容易，但通常并不是一个好选择。一个主要的原因是，one-hot词向量无法准确表达不同词之间的相似度，如我们常常使用的余弦相似度。对于向量$\boldsymbol{x}, \boldsymbol{y} \in \mathbb{R}^d$，它们的余弦相似度是它们之间夹角的余弦值
+ $\displaystyle\frac{\boldsymbol{x}^\top \boldsymbol{y}}{\|\boldsymbol{x}\| \|\boldsymbol{y}\|} \in [-1, 1]$

由于任何两个不同词的one-hot向量的余弦相似度都为0，多个不同词之间的相似度难以通过one-hot向量准确地体现出来。

word2vec工具的提出正是为了解决上面这个问题。它将每个词表示成一个定长的向量，并使得这些向量能较好地表达不同词之间的相似和类比关系。word2vec工具包含了两个模型，即`跳字模型`(skip-gram)和`连续词袋模型`(continuous bag of words，CBOW)。接下来让我们分别介绍这两个模型以及它们的训练方法。

### 10.1.2 跳字模型
> 原文比较简单，参考我整理的文档

跳字模型假设基于某个词来生成它在文本序列周围的词。举个例子，假设文本序列是“the”、“man”、“loves”、“his”、“son”。以“loves”作为中心词，设背景窗口大小为2。如`图10.1`所示，跳字模型所关心的是，给定中心词“loves”，生成与它距离不超过2个词的背景词“the”、“man”、“his”、“son”的条件概率，即
+ $P(\textrm{'the'},\textrm{'man'},\textrm{'his'},\textrm{'son'}\mid\textrm{'loves'})$

假设给定中心词的情况下，背景词的生成是相互独立的，那么上式可以改写成
+ $P(\textrm{'the'}\mid\textrm{'loves'})\cdot P(\textrm{'man'}\mid\textrm{'loves'})\cdot P(\textrm{'his'}\mid\textrm{'loves'})\cdot P(\textrm{'son'}\mid\textrm{'loves'})$

<img src="images/10_01.png" style="width:350px;"/>

在跳字模型中，每个词被表示成两个$d$维向量，用来计算条件概率。假设这个词在词典中索引为$i$，当它为中心词时向量表示为$\boldsymbol{v}_i\in\mathbb{R}^d$，而为背景词时向量表示为$\boldsymbol{u}_i\in\mathbb{R}^d$。设中心词$w_c$在词典中索引为$c$，背景词$w_o$在词典中索引为$o$，给定中心词生成背景词的条件概率可以通过对向量内积做softmax运算而得到：
+ $\displaystyle P(w_o \mid w_c) = \frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}$

其中词典索引集$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。假设给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$。假设给定中心词的情况下背景词的生成相互独立，当背景窗口大小为$m$时，跳字模型的似然函数即给定任一中心词生成所有背景词的概率
+ $\displaystyle \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)})$

这里小于1或大于$T$的时间步可以被忽略。

##### 训练跳字模型
跳字模型的参数是每个词所对应的中心词向量和背景词向量。训练中我们通过最大化似然函数来学习模型参数，即最大似然估计。这等价于最小化以下损失函数：
+ $\displaystyle - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)})$


如果使用随机梯度下降，那么在每一次迭代里我们随机采样一个较短的子序列来计算有关该子序列的损失，然后计算梯度来更新模型参数。梯度计算的关键是条件概率的对数有关中心词向量和背景词向量的梯度。根据定义，首先看到
+ $\displaystyle\log P(w_o \mid w_c) =
\boldsymbol{u}_o^\top \boldsymbol{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right)$

通过微分，我们可以得到上式中$\boldsymbol{v}_c$的梯度
$$
\begin{aligned}
\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \boldsymbol{v}_c} 
&= \boldsymbol{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\boldsymbol{u}_j^\top \boldsymbol{v}_c)\boldsymbol{u}_j}{\sum_{i \in \mathcal{V}} \exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\\
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\boldsymbol{u}_j^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\right) \boldsymbol{u}_j\\ 
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \boldsymbol{u}_j.
\end{aligned}
$$

它的计算需要词典中所有词以$w_c$为中心词的条件概率。有关其他词向量的梯度同理可得。

训练结束后，对于词典中的任一索引为$i$的词，我们均得到该词作为中心词和背景词的两组词向量$\boldsymbol{v}_i$和$\boldsymbol{u}_i$。在自然语言处理应用中，一般使用跳字模型的中心词向量作为词的表征向量。

## 10.1.3 连续词袋模型
连续词袋模型与跳字模型类似。与跳字模型最大的不同在于，连续词袋模型假设基于某中心词在文本序列前后的背景词来生成该中心词。在同样的文本序列`the`、`man`、`loves`、`his`、`son`里，以`loves`作为中心词，且背景窗口大小为2时，连续词袋模型关心的是，给定背景词`the`、`man`、`his`、`son`生成中心词`loves`的条件概率(如`图10.2`所示)，也就是
+ $P(\textrm{'loves'}\mid\textrm{'the'},\textrm{'man'},\textrm{'his'},\textrm{'son'})$

<img src="images/10_02.png" style="width:400px;"/>

因为连续词袋模型的背景词有多个，我们将这些背景词向量取平均，然后使用和跳字模型一样的方法来计算条件概率。设$\boldsymbol{v_i}\in\mathbb{R}^d$和$\boldsymbol{u_i}\in\mathbb{R}^d$分别表示词典中索引为$i$的词作为背景词和中心词的向量(注意符号的含义与跳字模型中的相反)。设中心词$w_c$在词典中索引为$c$，背景词$w_{o_1}, \ldots, w_{o_{2m}}$在词典中索引为$o_1, \ldots, o_{2m}$，那么给定背景词生成中心词的条件概率
+ $P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\boldsymbol{u}_c^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\boldsymbol{u}_i^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}$

我们记$\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$，且$\displaystyle\bar{\boldsymbol{v}}_o = \frac{\left(\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}} \right)}{(2m)}$，那么上式可以简写成
+ $\boldsymbol{P}(w_c \mid \mathcal{W}_o) = \displaystyle\frac{\exp\left(\boldsymbol{u}_c^\top \bar{\boldsymbol{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o\right)}$

给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$，背景窗口大小为$m$。连续词袋模型的似然函数是由背景词生成任一中心词的概率
+ $ \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots,  w^{(t-1)},  w^{(t+1)}, \ldots,  w^{(t+m)})$

##### 训练连续词袋模型
训练连续词袋模型同训练跳字模型基本一致。连续词袋模型的最大似然估计等价于最小化损失函数
+ $  -\sum_{t=1}^T  \text{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots,  w^{(t-1)},  w^{(t+1)}, \ldots,  w^{(t+m)})$

注意到
+ $\log\,P(w_c \mid \mathcal{W}_o) = \boldsymbol{u}_c^\top \bar{\boldsymbol{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o\right)\right)$

通过微分，我们可以计算出上式中条件概率的对数有关任一背景词向量$\boldsymbol{v}_{o_i}$（$i = 1, \ldots, 2m$）的梯度
+ $\displaystyle \frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \boldsymbol{v}_{o_i}} = \frac{1}{2m} \left(\boldsymbol{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\boldsymbol{u}_j^\top \bar{\boldsymbol{v}}_o)\boldsymbol{u}_j}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o)} \right) = \frac{1}{2m}\left(\boldsymbol{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \boldsymbol{u}_j \right)$

有关其他词向量的梯度同理可得。同跳字模型不一样的一点在于，我们一般使用连续词袋模型的背景词向量作为词的表征向量。

## 10.2 近似训练
回忆上一节的内容。跳字模型的核心在于使用softmax运算得到给定中心词$w_c$来生成背景词$w_o$的条件概率
+ $\boldsymbol{P}(w_o \mid w_c) = \displaystyle\frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}$

该条件概率相应的对数损失
+ $-\log \boldsymbol{P}(w_o \mid w_c) = -\boldsymbol{u}_o^\top \boldsymbol{v}_c + \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right)$

由于softmax运算考虑了背景词可能是词典$\mathcal{V}$中的任一词，以上损失包含了词典大小数目的项的累加。在上一节中我们看到，不论是跳字模型还是连续词袋模型，由于条件概率使用了softmax运算，每一步的梯度计算都包含词典大小数目的项的累加。对于含几十万或上百万词的较大词典，每次的梯度计算开销可能过大。为了降低该计算复杂度，本节将介绍两种近似训练方法，即`负采样`(negative sampling)或`层序softmax`(hierarchical softmax)。

### 10.2.1 负采样
负采样修改了原来的目标函数。给定中心词$w_c$的一个背景窗口，我们把背景词$w_o$出现在该背景窗口看作一个事件，并将该事件的概率计算为
+ $\boldsymbol{P}(D=1\mid w_c, w_o) = \sigma(\boldsymbol{u}_o^\top \boldsymbol{v}_c)$

其中的$\sigma$函数与sigmoid激活函数的定义相同：
+ $\sigma(x) = \displaystyle\frac{1}{1+e^{-x}}$

我们先考虑最大化文本序列中所有该事件的联合概率来训练词向量。具体来说，给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$且背景窗口大小为$m$，考虑最大化联合概率
+ $ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)})$

然而，以上模型中包含的事件仅考虑了正类样本。这导致当所有词向量相等且值为无穷大时，以上的联合概率才被最大化为1。很明显，这样的词向量毫无意义。负采样通过采样并添加负类样本使目标函数更有意义。设背景词$w_o$出现在中心词$w_c$的一个背景窗口为事件$P$，我们根据分布$P(w)$采样$K$个未出现在该背景窗口中的词，即噪声词。设噪声词$w_k$（$k=1, \ldots, K$）不出现在中心词$w_c$的该背景窗口为事件$N_k$。假设同时含有正类样本和负类样本的事件$P, N_1, \ldots, N_K$相互独立，负采样将以上需要最大化的仅考虑正类样本的联合概率改写为
+ $ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)})$

其中条件概率被近似表示为 
+ $ P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k)$

设文本序列中时间步$t$的词$w^{(t)}$在词典中的索引为$i_t$，噪声词$w_k$在词典中的索引为$h_k$。有关以上条件概率的对数损失为
$$\displaystyle\begin{aligned} -\log P(w^{(t+j)} \mid w^{(t)}) =& -\log P(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\\ =&- \log\sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\boldsymbol{u}_{h_k}^\top \boldsymbol{v}{i_t}\right)\right)\\ =&- \log\sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right). \end{aligned} $$

现在，训练中每一步的梯度计算开销不再与词典大小相关，而与$K$线性相关。当$K$取较小的常数时，负采样在每一步的梯度计算开销较小。

### 10.2.2 层序softmax
层序softmax是另一种近似训练法。它使用了二叉树这一数据结构，树的每个叶结点代表词典$\mathcal{V}$中的每个词。

<img src="images/10_03.png" style="width:500px;"/>

假设$L(w)$为从二叉树的根结点到词$w$的叶结点的路径(包括根结点和叶结点)上的结点数。设$n(w,j)$为该路径上第$j$个结点，并设该结点的背景词向量为$\boldsymbol{u}_{n(w,j)}$。以图10.3为例，$L(w_3) = 4$。层序softmax将跳字模型中的条件概率近似表示为
+ $P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma\left( [![ n(w_o, j+1) = \text{leftChild}(n(w_o,j)) ]!] \cdot \boldsymbol{u}_{n(w_o,j)}^\top \boldsymbol{v}_c\right)$

其中$\sigma$函数与“多层感知机”一节中sigmoid激活函数的定义相同，$\text{leftChild}(n)$是结点$n$的左子结点：如果判断$x$为真，$[![x]!] = 1$；反之$[![x]!] = -1$。 让我们计算`图10.3`中给定词$w_c$生成词$w_3$的条件概率。我们需要将$w_c$的词向量$\boldsymbol{v}_c$和根结点到$w_3$路径上的非叶结点向量一一求内积。由于在二叉树中由根结点到叶结点$w_3$的路径上需要向左、向右再向左地遍历(`图10.3`中加粗的路径)，我们得到
+ $P(w_3 \mid w_c) = \sigma(\boldsymbol{u}_{n(w_3,1)}^\top \boldsymbol{v}_c) \cdot \sigma(-\boldsymbol{u}_{n(w_3,2)}^\top \boldsymbol{v}_c) \cdot \sigma(\boldsymbol{u}_{n(w_3,3)}^\top \boldsymbol{v}_c)$

由于$\sigma(x)+\sigma(-x) = 1$，给定中心词$w_c$生成词典$\mathcal{V}$中任一词的条件概率之和为1这一条件也将满足：
+ $\displaystyle\sum_{w \in \mathcal{V}} P(w \mid w_c) = 1$

此外，由于$L(w_o)-1$的数量级为$\displaystyle\mathcal{O}(\text{log}_2|\mathcal{V}|)$，当词典$\mathcal{V}$很大时，层序softmax在训练中每一步的梯度计算开销相较未使用近似训练时大幅降低。

## 10.3 word2vec的实现
本节是对前两节内容的实践。我们以负采样为例，介绍在语料库上训练词嵌入模型的实现。我们还会介绍一些实现中的技巧，如二次采样。

### 10.3.1 预处理数据集
PTB(Penn Tree Bank)是一个常用的小型语料库。它采样自《华尔街日报》的文章，包括训练集、验证集和测试集。该数据集的每一行作为一个句子。句子中的每个词由空格隔开。

In [2]:
with open('./data/ptb/ptb.train.txt', 'r') as f:
    lines = f.readlines() 
    # st是sentence的缩写 
    raw_dataset = [st.split() for st in lines]

'# sentences: %d' % len(raw_dataset)

'# sentences: 42068'

对于数据集的前3个句子，打印每个句子的词数和前5个词。这个数据集中句尾符为`<eos>`，生僻词全用`<unk>`表示，数字则被替换成了`N`。

In [3]:
for st in raw_dataset[:3]:
    print('# tokens:', len(st), st[:5])

# tokens: 24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
# tokens: 15 ['pierre', '<unk>', 'N', 'years', 'old']
# tokens: 11 ['mr.', '<unk>', 'is', 'chairman', 'of']


##### 建⽴词语索引
为了计算简单，我们只保留在数据集中⾄少出现5次的词，然后将词映射到整数索引：

In [4]:
# tk是token的缩写
counter = collections.Counter([tk for st in raw_dataset for tk in st]) 
counter = dict(filter(lambda x: x[1] >= 5, counter.items()))

idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)}

dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset] 
num_tokens = sum([len(st) for st in dataset]) 
'# tokens: %d' % num_tokens

'# tokens: 887100'

##### 二次采样
文本数据中一般会出现一些高频词，如英文中的`the`、`a`和`in`。通常来说，在一个背景窗口中，一个词(如`chip`)和较低频词(如`microprocessor`)同时出现比和高频词(如`the`)同时出现对训练词嵌入模型更有益。因此，训练词嵌入模型时可以对词进行二次采样(subsampling)。 具体来说，数据集中每个被索引词$w_i$将有一定概率被丢弃，该丢弃概率为
+ $\displaystyle P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right)$

其中$f(w_i)$是数据集中词$w_i$的个数与总词数之比，常数$t$是一个超参数(实验中设为$10^{-4}$)。可见，只有当$f(w_i) > t$时，我们才有可能在二次采样中丢弃词$w_i$，并且越高频的词被丢弃的概率越大。

In [5]:
def discard(idx):
    return random.uniform(0, 1) < 1 - math.sqrt( 1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset] 
'# tokens: %d' % sum([len(st) for st in subsampled_dataset])

'# tokens: 375772'

可以看到，⼆次采样后我们去掉了⼀半左右的词。下⾯⽐较⼀个词在⼆次采样前后出现在数据集中的次数。可⻅⾼频词`the`的采样率不⾜1/20。

In [6]:
def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, 
        sum( [st.count(token_to_idx[token]) for st in dataset]), 
        sum( [st.count(token_to_idx[token]) for st in subsampled_dataset]))

compare_counts('the')

'# the: before=50770, after=2130'

但低频词`join`则完整地保留了下来。

In [7]:
compare_counts('join')

'# join: before=45, after=45'

##### 提取中⼼词和背景词
我们将与中⼼词距离不超过背景窗口⼤小的词作为它的背景词。下⾯定义函数提取出所有中⼼词和它们的背景词。它每次在整数1和`max_window_size`(最⼤背景窗口)之间随机均匀采样⼀个整数作为背景窗口⼤小。

In [8]:
def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [], [] 
    for st in dataset:
        if len(st) < 2: 
            # 每个句⼦⾄少要有2个词才可能组成⼀对“中⼼词-背景词” 
            continue 
        centers += st 
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size) 
            indices = list(range(max(0, center_i - window_size), min(len(st), center_i + 1 + window_size)))
            # 将中⼼词排除在背景词之外
            indices.remove(center_i) 
            contexts.append([st[idx] for idx in indices]) 
    return centers, contexts

下⾯我们创建⼀个⼈⼯数据集，其中含有词数分别为7和3的两个句⼦。设最⼤背景窗口为2，打印所有中⼼词和它们的背景词。

In [9]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset) 

for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1, 2]
center 1 has contexts [0, 2, 3]
center 2 has contexts [1, 3]
center 3 has contexts [1, 2, 4, 5]
center 4 has contexts [3, 5]
center 5 has contexts [4, 6]
center 6 has contexts [4, 5]
center 7 has contexts [8]
center 8 has contexts [7, 9]
center 9 has contexts [8]


实验中，我们设最⼤背景窗口⼤小为5。下⾯提取数据集中所有的中⼼词及其背景词。

In [10]:
all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

### 10.3.2 负采样
我们使⽤负采样来进⾏近似训练。对于⼀对中⼼词和背景词，我们随机采样$K$个噪声词(实验中设$K = 5$)。根据word2vec论⽂的建议，噪声词采样概率$P(w)$设为$w$词频与总词频之⽐的0.75次⽅。

In [11]:
def get_negatives(all_contexts, sampling_weights, K):
    all_negatives, neg_candidates, i = [], [], 0 
    population = list(range(len(sampling_weights))) 
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # 根据每个词的权重(sampling_weights)随机⽣成k个词的索引作为噪声词
                # 为了⾼效计算，可以将k设得稍⼤⼀点 
                i, neg_candidates = 0, random.choices(population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            # 噪声词不能是背景词 
            if neg not in set(contexts): 
                negatives.append(neg) 
        all_negatives.append(negatives) 
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token] 
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

### 10.3.3 读取数据集
我们从数据集中提取所有中心词`all_centers`，以及每个中心词对应的背景词`all_contexts`和噪声词`all_negatives`。我们将通过随机小批量来读取它们。

在一个小批量数据中，第$i$个样本包括一个中心词以及它所对应的$n_i$个背景词和$m_i$个噪声词。由于每个样本的背景窗口大小可能不一样，其中背景词与噪声词个数之和$n_i+m_i$也会不同。在构造小批量时，我们将每个样本的背景词和噪声词连结在一起，并添加填充项0直至连结后的长度相同，即长度均为$\max_i n_i+m_i$(max_len变量)。为了避免填充项对损失函数计算的影响，我们构造了掩码变量`masks`，其每一个元素分别与连结后的背景词和噪声词`contexts_negatives`中的元素一一对应。当`contexts_negatives`变量中的某个元素为填充项时，相同位置的掩码变量`masks`中的元素取0，否则取1。为了区分正类和负类，我们还需要将`contexts_negatives`变量中的背景词和噪声词区分开来。依据掩码变量的构造思路，我们只需创建与`contexts_negatives`变量形状相同的标签变量`labels`，并将与背景词(正类)对应的元素设1，其余清0。

下面我们实现这个小批量读取函数`batchify`。它的小批量输入`data`是一个长度为批量大小的列表，其中每个元素分别包含中心词`center`、背景词`context`和噪声词`negative`。该函数返回的小批量数据符合我们需要的格式，例如，包含了掩码变量。

In [12]:
def batchify(data):
    max_len = max(len(c) + len(n) for _, c, n in data) 
    centers, contexts_negatives, masks, labels = [], [], [], [] 
    for center, context, negative in data:
        cur_len = len(context) + len(negative) 
        centers += [center] 
        contexts_negatives += [context + negative + [0] * (max_len - cur_len)] 
        masks += [[1] * cur_len + [0] * (max_len - cur_len)] 
        labels += [[1] * len(context) + [0] * (max_len - len(context))] 
    return (nd.array(centers).reshape((-1, 1)), nd.array(contexts_negatives), nd.array(masks), nd.array(labels))

我们⽤刚刚定义的`batchify`函数指定`DataLoader`实例中小批量的读取⽅式，然后打印读取的第⼀个批量中各个变量的形状。

In [13]:
batch_size = 512
num_workers = 4 

dataset = gdata.ArrayDataset(all_centers, all_contexts, all_negatives) 
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True, batchify_fn=batchify, num_workers=num_workers) 

for batch in data_iter:
    for name, data in zip(['centers', 'contexts_negatives', 'masks', 'labels'], batch):
        print(name, 'shape:', data.shape)
    break

centers shape: (512, 1)
contexts_negatives shape: (512, 60)
masks shape: (512, 60)
labels shape: (512, 60)


### 10.3.4 跳字模型
我们将通过使用嵌入层和小批量乘法来实现跳字模型。它们也常常用于实现其他自然语言处理的应用。

##### 嵌入层
获取词嵌入的层称为`嵌入层`，在`Gluon`中可以通过创建`nn.Embedding`实例得到。嵌入层的权重是一个矩阵，其行数为词典大小(input_dim)，列数为每个词向量的维度(output_dim)。我们设词典大小为20，词向量的维度为4。

In [14]:
embed = nn.Embedding(input_dim=20, output_dim=4) 
embed.initialize() 
embed.weight

Parameter embedding0_weight (shape=(20, 4), dtype=float32)

嵌⼊层的输⼊为词的索引。输⼊⼀个词的索引i，嵌⼊层返回权重矩阵的第$i$⾏作为它的词向量。下⾯我们将形状为$(2, 3)$的索引输⼊进嵌⼊层，由于词向量的维度为4，我们得到形状为(2, 3, 4)的词向量。

In [15]:
x = nd.array([[1, 2, 3], [4, 5, 6]]) 
embed(x)


[[[ 0.01438687  0.05011239  0.00628365  0.04861524]
  [-0.01068833  0.01729892  0.02042518 -0.01618656]
  [-0.00873779 -0.02834515  0.05484822 -0.06206018]]

 [[ 0.06491279 -0.03182812 -0.01631819 -0.00312688]
  [ 0.0408415   0.04370362  0.00404529 -0.0028032 ]
  [ 0.00952624 -0.01501013  0.05958354  0.04705103]]]
<NDArray 2x3x4 @cpu(0)>

##### 小批量乘法
我们可以使用小批量乘法运算`batch_dot`对两个小批量中的矩阵一一做乘法。假设第一个小批量包含$n$个形状为$a\times b$的矩阵$\boldsymbol{X}_1, \ldots, \boldsymbol{X}_n$，第二个小批量包含$n$个形状为$b\times c$的矩阵$\boldsymbol{Y}_1, \ldots, \boldsymbol{Y}_n$。这两个小批量的矩阵乘法输出为$n$个形状为$a\times c$的矩阵$\boldsymbol{X}_1\boldsymbol{Y}_1, \ldots, \boldsymbol{X}_n\boldsymbol{Y}_n$。因此，给定两个形状分别为($n$, $a$, $b$)和($n$, $b$, $c$)的NDArray，小批量乘法输出的形状为($n$, $a$, $c$)。

In [16]:
X = nd.ones((2, 1, 4))
Y = nd.ones((2, 4, 6)) 
nd.batch_dot(X, Y).shape
# print(X)
# print(Y)
# print(nd.batch_dot(X,Y))

(2, 1, 6)

##### 跳字模型前向计算
在前向计算中，跳字模型的输入包含中心词索引`center`以及连结的背景词与噪声词索引`contexts_and_negatives`。其中`center`变量的形状为`(批量大小, 1)`，而`contexts_and_negatives`变量的形状为`(批量大小, max_len)`。这两个变量先通过词嵌入层分别由词索引变换为词向量，再通过小批量乘法得到形状为`(批量大小, 1, max_len)`的输出。输出中的每个元素是中心词向量与背景词向量或噪声词向量的内积。

In [17]:
def skip_gram(center, contexts_and_negatives, embed_v, embed_u): 
    v = embed_v(center) 
    u = embed_u(contexts_and_negatives) 
    pred = nd.batch_dot(v, u.swapaxes(1, 2)) 
    return pred

### 10.3.5 训练模型
在训练词嵌⼊模型之前，我们需要定义模型的损失函数。

##### 二元交叉熵损失函数
根据负采样中损失函数的定义，我们可以直接使用`Gluon`的二元交叉熵损失函数`SigmoidBinaryCrossEntropyLoss`。

In [18]:
loss = gloss.SigmoidBinaryCrossEntropyLoss()

值得一提的是，我们可以通过掩码变量指定小批量中参与损失函数计算的部分预测值和标签：当掩码为1时，相应位置的预测值和标签将参与损失函数的计算；当掩码为0时，相应位置的预测值和标签则不参与损失函数的计算。我们之前提到，掩码变量可用于避免填充项对损失函数计算的影响。

In [19]:
pred = nd.array([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]]) 
# 标签变量label中的1和0分别代表背景词和噪声词
label = nd.array([[1, 0, 0, 0], [1, 1, 0, 0]])
#掩码变量
mask = nd.array([[1, 1, 1, 1], [1, 1, 1, 0]]) 
loss(pred, label, mask) * mask.shape[1] / mask.sum(axis=1)


[0.8739896 1.2099689]
<NDArray 2 @cpu(0)>

作为⽐较，下⾯将从零开始实现⼆元交叉熵损失函数的计算，并根据掩码变量`mask`计算掩码为1的预测值和标签的损失。

In [20]:
def sigmd(x):
    return -math.log(1 / (1 + math.exp(-x)))

print('%.7f' % ((sigmd(1.5) + sigmd(-0.3) + sigmd(1) + sigmd(-2)) / 4)) 
print('%.7f' % ((sigmd(1.1) + sigmd(-0.6) + sigmd(-2.2)) / 3))

0.8739896
1.2099689


##### 初始化模型参数
我们分别构造中⼼词和背景词的嵌⼊层，并将超参数词向量维度`embed_size`设置成100。

In [21]:
embed_size = 100
net = nn.Sequential() 
net.add(nn.Embedding(input_dim=len(idx_to_token), output_dim=embed_size),
        nn.Embedding(input_dim=len(idx_to_token), output_dim=embed_size))

##### 定义训练函数
下⾯定义训练函数。由于填充项的存在，与之前的训练函数相⽐，损失函数的计算稍有不同。

In [22]:
def train(net, lr, num_epochs):
    # ctx = d2l.try_gpu() 
    ctx = mx.cpu()
    net.initialize(ctx=ctx, force_reinit=True) 
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr}) 
    for epoch in range(num_epochs):
        start, l_sum, n = time(), 0.0, 0
        for batch in data_iter:
            center, context_negative, mask, label = [ data.as_in_context(ctx) for data in batch]
            with autograd.record():
                pred = skip_gram(center, context_negative, net[0], net[1])
                # 使⽤掩码变量mask来避免填充项对损失函数计算的影响
                l = (loss(pred.reshape(label.shape), label, mask) * mask.shape[1] / mask.sum(axis=1)) 
            l.backward() 
            trainer.step(batch_size) 
            l_sum += l.sum().asscalar() 
            n += l.size 
        print('epoch %d, loss %.2f, time %.2fs' % (epoch + 1, l_sum / n, time() - start))

现在我们就可以使⽤负采样训练跳字模型了。

In [23]:
train(net, 0.005, 5)

epoch 1, loss 0.46, time 146.46s
epoch 2, loss 0.39, time 147.39s
epoch 3, loss 0.35, time 146.85s
epoch 4, loss 0.32, time 148.37s
epoch 5, loss 0.31, time 147.53s


### 10.3.6 应用词嵌入模型
训练好词嵌入模型之后，我们可以根据两个词向量的余弦相似度表示词与词之间在语义上的相似度。可以看到，使用训练得到的词嵌入模型时，与词`chip`语义最接近的词大多与芯片有关。

In [24]:
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data() 
    x = W[token_to_idx[query_token]] 
    # 添加的1e-9是为了数值稳定性 
    cos = nd.dot(W, x) / (nd.sum(W * W, axis=1) * nd.sum(x * x) + 1e-9).sqrt() 
    topk = nd.topk(cos, k=k+1, ret_typ='indices').asnumpy().astype('int32')
    for i in topk[1:]: # 除去输⼊词 
        print('cosine sim=%.3f: %s' % (cos[i].asscalar(), (idx_to_token[i])))

get_similar_tokens('chip', 3, net[0])

cosine sim=0.567: computers
cosine sim=0.561: intel
cosine sim=0.560: computer


## 10.4 子词嵌入(fastText)
英语单词通常有其内部结构和形成方式。例如，我们可以从`dog`、`dogs`和`dogcatcher`的字面上推测它们的关系。这些词都有同一个词根`dog`，但使用不同的后缀来改变词的含义。

这个关联也可以推广至其他词汇。例如，`dog`和`dogs`的关系如同`cat`和`cats`的关系，`boy`和`boyfriend`的关系如同`girl`和`girlfriend`的关系。这一特点并非为英语所独有。在法语和西班牙语中，很多动词根据场景不同有40多种不同的形态，而在芬兰语中，一个名词可能有15种以上的形态。事实上，构词学(morphology)作为语言学的一个重要分支，研究的正是词的内部结构和形成方式。

在word2vec中，我们并没有直接利用构词学中的信息。无论是在跳字模型还是连续词袋模型中，我们都将形态不同的单词用不同的向量来表示。例如，`dog`和`dogs`分别用两个不同的向量表示，而模型中并未直接表达这两个向量之间的关系。鉴于此，fastText提出了`子词嵌入`(subword embedding)的方法，从而试图将构词信息引入word2vec中的跳字模型。

在fastText中，每个中心词被表示成子词的集合。下面我们用单词`where`作为例子来了解子词是如何产生的。首先，我们在单词的首尾分别添加特殊字符`<`和`>`以区分作为前后缀的子词。然后，将单词当成一个由字符构成的序列来提取$n$元语法。例如，当$n=3$时，我们得到所有长度为3的子词：`<wh`、`whe`、`her`、`ere`、`re>`以及特殊子词`<where>`。

在fastText中，对于一个词$w$，我们将它所有长度在$3 \sim 6$的子词和特殊子词的并集记为$\mathcal{G}_w$。那么词典则是所有词的子词集合的并集。假设词典中子词$g$的向量为$\boldsymbol{z}_g$，那么跳字模型中词$w$的作为中心词的向量$\boldsymbol{v}_w$则表示成
+ $\boldsymbol{v}_w = \displaystyle\sum_{g\in\mathcal{G}_w} \boldsymbol{z}_g$

fastText的其余部分同跳字模型一致，不在此重复。可以看到，与跳字模型相比，fastText中词典规模更大，造成模型参数更多，同时一个词的向量需要对所有子词向量求和，继而导致计算复杂度更高。但与此同时，较生僻的复杂单词，甚至是词典中没有的单词，可能会从同它结构类似的其他词那里获取更好的词向量表示。

## 10.5 全局向量的词嵌⼊(GloVe)
让我们先回顾一下word2vec中的跳字模型。将跳字模型中使用softmax运算表达的条件概率$P(w_j\mid w_i)$记作$q_{ij}$，即
+ $\displaystyle q_{ij}=\frac{\exp(\boldsymbol{u}_j^\top \boldsymbol{v}_i)}{ \sum_{k \in \mathcal{V}} \text{exp}(\boldsymbol{u}_k^\top \boldsymbol{v}_i)}$

其中$\boldsymbol{v}_i$和$\boldsymbol{u}_i$分别是索引为$i$的词$w_i$作为中心词和背景词时的向量表示，$\mathcal{V} = {0, 1, \ldots, |\mathcal{V}|-1}$为词典索引集。

对于词$w_i$，它在数据集中可能多次出现。我们将每一次以它作为中心词的所有背景词全部汇总并保留重复元素，记作`多重集`(multiset)$\mathcal{C}_i$。一个元素在多重集中的个数称为该元素的`重数`(multiplicity)。

举例来说，假设词$w_i$在数据集中出现2次：文本序列中以这2个$w_i$作为中心词的背景窗口分别包含背景词索引$2,1,5,2$和$2,3,2,1$。那么多重集$\mathcal{C}_i = {1,1,2,2,2,2,3,5}$，其中元素1的重数为2，元素2的重数为4，元素3和5的重数均为1。将多重集$\mathcal{C}_i$中元素$j$的重数记作$x_{ij}$：它表示了整个数据集中所有以$w_i$为中心词的背景窗口中词$w_j$的个数。那么，跳字模型的损失函数还可以用另一种方式表达：
+ $\displaystyle -\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}} x_{ij} \log(q_{ij})$

我们将数据集中所有以词$w_i$为中心词的背景词的数量之和$\left|\mathcal{C}_i\right|$记为$x_i$，并将以$w_i$为中心词生成背景词$w_j$的条件概率$\displaystyle \frac{x_{ij}}{x_i}$记作$p_{ij}$。我们可以进一步将跳字模型的损失函数改写为
+ $\displaystyle -\sum_{i\in\mathcal{V}} x_i \sum_{j\in\mathcal{V}} p_{ij} \log(q_{ij})$

上式中，$\displaystyle -\sum_{j\in\mathcal{V}} p_{ij} \log(q_{ij})$计算的是以$w_i$为中心词的背景词条件概率分布$p_{ij}$和模型预测的条件概率分布$q_{ij}$的交叉熵，且损失函数使用所有以词$w_i$为中心词的背景词的数量之和来加权。最小化上式中的损失函数会令预测的条件概率分布尽可能接近真实的条件概率分布。

然而，作为常用损失函数的一种，交叉熵损失函数有时并不是好的选择。一方面，正如我们在“近似训练”一节中所提到的，令模型预测$q_{ij}$成为合法概率分布的代价是它在分母中基于整个词典的累加项。这很容易带来过大的计算开销。另一方面，词典中往往有大量生僻词，它们在数据集中出现的次数极少。而有关大量生僻词的条件概率分布在交叉熵损失函数中的最终预测往往并不准确。

### 10.5.1 GloVe模型
鉴于此，作为在word2vec之后提出的词嵌入模型，GloVe模型采用了平方损失，并基于该损失对跳字模型做了3点改动：
1. 使用非概率分布的变量$p'_{ij}=x_{ij}$和$q'_{ij}=\exp(\boldsymbol{u}_j^\top \boldsymbol{v}_i)$，并对它们取对数。因此，平方损失项是$\left(\log(p'_{ij}) - \log(q'_{ij})\right)^2 = \left(\boldsymbol{u}_j^\top \boldsymbol{v}_i - \log(x{ij})\right)^2$
2. 为每个词$w_i$增加两个为标量的模型参数：中心词偏差项$b_i$和背景词偏差项$c_i$
3. 将每个损失项的权重替换成函数$h(x_{ij})$。权重函数$h(x)$是值域在$[0,1]$的单调递增函数

如此一来，GloVe模型的目标是最小化损失函数
+ $\displaystyle \sum_{i\in\mathcal{V}} \sum_{j\in\mathcal{V}} h(x_{ij}) \left(\boldsymbol{u}_j^\top \boldsymbol{v}_i + b_i + c_j - \log(x_{ij})\right)^2$

其中权重函数$h(x)$的一个建议选择是：
+ 当$x < c$时(如$c = 100$)，令$h(x) = (\displaystyle\frac{x}{c})^\alpha$(如$\alpha = 0.75$)
+ 反之令$h(x) = 1$

因为$h(0)=0$，所以对于$x_{ij}=0$的平方损失项可以直接忽略。当使用小批量随机梯度下降来训练时，每个时间步我们随机采样小批量非零$x_{ij}$，然后计算梯度来迭代模型参数。这些非零$x_{ij}$是预先基于整个数据集计算得到的，包含了数据集的全局统计信息。因此，GloVe模型的命名取`全局向量`(Global Vectors)之意。

需要强调的是，如果词$w_i$出现在词$w_j$的背景窗口里，那么词$w_j$也会出现在词$w_i$的背景窗口里。也就是说，$x_{ij}=x_{ji}$。不同于word2vec中拟合的是非对称的条件概率$p_{ij}$，GloVe模型拟合的是对称的$\log(x_{ij})$。因此，任意词的中心词向量和背景词向量在GloVe模型中是等价的。但由于初始化值的不同，同一个词最终学习到的两组词向量可能不同。当学习得到所有词向量以后，GloVe模型使用中心词向量与背景词向量之和作为该词的最终词向量。

### 10.5.2 从条件概率比值理解GloVe模型
我们还可以从另外一个角度来理解GloVe模型。沿用本节前面的符号，$P(w_j \mid w_i)$表示数据集中以$w_i$为中心词生成背景词$w_j$的条件概率，并记作$p_{ij}$。作为源于某大型语料库的真实例子，以下列举了两组分别以`ice`和`steam`为中心词的条件概率以及它们之间的比值：

<img src="images/tbl_10_01.png" style="width:500px;"/>

我们可以观察到以下现象：
+ 对于与`ice`相关而与`steam`不相关的词$w_k$，如$w_k=`solid`$，我们期望条件概率比值较大，如上表最后一行中的值8.9
+ 对于与`ice`不相关而与`steam`相关的词$w_k$，如$w_k=`gas`$，我们期望条件概率比值较小，如上表最后一行中的值0.085
+ 对于与`ice`和`steam`都相关的词$w_k$，如$w_k=`water`$，我们期望条件概率比值接近1，如上表最后一行中的值1.36
+ 对于与`ice`和`steam`都不相关的词$w_k$，如$w_k=`fashion`$，我们期望条件概率比值接近1，如上表最后一行中的值0.96

由此可见，条件概率比值能比较直观地表达词与词之间的关系。我们可以构造一个词向量函数使它能有效拟合条件概率比值。我们知道，任意一个这样的比值需要3个词$w_i$、$w_j$和$w_k$。以$w_i$作为中心词的条件概率比值为$\displaystyle\frac{{p_{ij}}}{{p_{ik}}}$。我们可以找一个函数，它使用词向量来拟合这个条件概率比值：
+ $\displaystyle f(\boldsymbol{u}_j, \boldsymbol{u}_k, {\boldsymbol{v}}_i) \approx \frac{p_{ij}}{p_{ik}}$

这里函数$f$可能的设计并不唯一，我们只需考虑一种较为合理的可能性。注意到条件概率比值是一个标量，我们可以将$f$限制为一个标量函数：$f(\boldsymbol{u}_j, \boldsymbol{u}_k, {\boldsymbol{v}}_i) = f\left((\boldsymbol{u}_j - \boldsymbol{u}_k)^\top {\boldsymbol{v}}_i\right)$。交换索引$j$和$k$后可以看到函数$f$应该满足$f(x)f(-x)=1$，因此一种可能是$f(x)=\exp(x)$，于是
+ $f(\boldsymbol{u}_j, \boldsymbol{u}_k, {\boldsymbol{v}}_i) = \displaystyle\frac{\exp\left(\boldsymbol{u}_j^\top {\boldsymbol{v}}_i\right)}{\exp\left(\boldsymbol{u}_k^\top {\boldsymbol{v}}_i\right)} \approx \frac{p_{ij}}{p_{ik}}$

满足最右边约等号的一种可能是$\exp\left(\boldsymbol{u}_j^\top {\boldsymbol{v}}_i\right) \approx \alpha p_{ij}$，这里$\alpha$是一个常数。考虑到$p_{ij}=\displaystyle \frac{x_{ij}}{x_i}$，取对数后$\boldsymbol{u}_j^\top {\boldsymbol{v}}_i \approx \log(\alpha) + \log(x_{ij}) - \log(x_i)$。我们使用额外的偏差项来拟合$-\log(\alpha) + \log(x_i)$，例如，中心词偏差项$b_i$和背景词偏差项$c_j$：
+ $\boldsymbol{u}_j^\top \boldsymbol{v}_i + b_i + c_j \approx \log(x_{ij})$

对上式左右两边取平方误差并加权，我们可以得到GloVe模型的损失函数。